# Analiza Wielowymiarowa - zajecia 05 - Skalowanie wielowymiarowe i obserwacje nietypowe

## Skalowanie wielowymiarowe

In [ ]:
from multidim.utils import resolve_stata, load_stata

STATA_PATH, STATA_TYPE = resolve_stata(version = 18, stype = "se")
# make sure they are proper ones
STATA_PATH, STATA_TYPE

In [ ]:
#load_stata(STATA_PATH, STATA_TYPE)
load_stata(STATA_PATH, "se")

In [ ]:
import pandas as pd
import scipy
import sklearn
import numpy as np
import seaborn as sns
import scipy as sp
from scipy.stats import chi2
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from multidim.funs import plot_dendrogram
from multidim.datasets import load_boston

### Przykład 1.

Dane i przyklad zostaly pozyczone z podrecznika Stata MV

Zaladowanie danych z internetu. Jezeli nie dziala, zbior danych jest w materialach do zajec

Dane dotycza charakterystyk odzywczych platkow sniadanowych

In [ ]:
%%stata 
use https://www.stata-press.com/data/r17/cerealnut
des 

Podstawowy opis statystyczny cech charakterystycznych płatkow sniadaniowych

In [ ]:
%stata summarize calories-K, sep(4)

In [ ]:
%stata replace brand = subinstr(brand," ","_",.)

Polecenie mds wykonuje skalowanie wielowymiarowe na podstawie podanych charakterystyk
opcja "id" jest niezbedna, identyfikuje obserwacje

In [ ]:
%stata mds calories-K, id(brand)

Dwie pierwsze wartosci wlasne wyjasniaja 99,7% niepodobienstwa  

In [ ]:
%stata mds calories-K, id(brand) std

opcja "std" przed wykonaniem obliczen standaryzuje wartosci charakterystyk

## Obserwacje nietypowe

Wczytanie danych  do pakietu Stata

In [ ]:
%stata load boston 

Wczytanie danych

In [ ]:
boston = load_boston()
y = boston['MEDV'] 
df = boston[['CRIM', 'ZN', 'INDUS', 'CHAS','NOX','RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT' ]]

In [ ]:
print(boston)


In [ ]:
#df.describe()
df_1 = df[['TAX', 'B']]
df_2 = df[['CRIM', 'ZN', 'INDUS', 'RM', 'AGE', 'DIS', 'RAD', 'PTRATIO','LSTAT']]
df_3 = df[['CHAS', 'NOX']]

ax = sns.boxplot(data=df_2, orient="h", palette="Set2")

ax = sns.boxplot(x=df["CRIM"])
ax.set_xlabel('Crime rate per capita')


Wizualizacja danych

In [ ]:
#Wykres rozrzutu
ax = sns.scatterplot(x="LSTAT", y="CRIM", data=df)

Odległość Mahalanobisa - funckcja zwracająca obserwacje odstające

In [ ]:
def mahalanobis_method(df):
    #M-Distance
    x_minus_mu = df - np.mean(df, axis=0)
    cov = np.cov(df.values.T)                           #Covariance
    inv_covmat = sp.linalg.inv(cov)                     #Inverse covariance
    left_term = np.dot(x_minus_mu, inv_covmat) 
    mahal = np.dot(left_term, x_minus_mu.T)
    md = np.sqrt(mahal.diagonal())
    
    #Flag as outlier
    outlier = []
    #Cut-off point
    C = np.sqrt(chi2.ppf((1-0.001), df=df.shape[1]))    #degrees of freedom = number of variables
    for index, value in enumerate(md):
        if value > C:
            outlier.append(index)
        else:
            continue
    return outlier, md

Obserwacje odstające według odległości Mahalanobisa: 2 wymiarowe

In [ ]:
df_bivariate = df[['LSTAT', 'CRIM']]

In [ ]:
outliers_mahal_bi, md_bi = mahalanobis_method(df=df_bivariate)

In [ ]:
outliers_mahal_bi

Obserwacje odstające według odległości Mahalanobisa: względem wszystkich zmiennych w zbiorze

In [ ]:
outliers_mahal, md = mahalanobis_method(df=df)

In [ ]:
outliers_mahal

### Winsoryzacja

Stworzenie kopii danych

In [ ]:
df_win = df.copy(deep=True)

Winsoryzacja prawego ogona rozkładu 

In [ ]:
df_win['CRIM_wins_95%'] = sp.stats.mstats.winsorize(df['CRIM'], limits=(0, 0.05))
df_win['CRIM_wins_925%'] = sp.stats.mstats.winsorize(df['CRIM'], limits=(0, 0.075))

Tu będzie opis

In [ ]:
df_win.describe()

Wykres rozkładu zmiennej Crime rate per capita by town

In [ ]:
sns.distplot(df['CRIM'])

Wykresy po winsoryzacji 

In [ ]:
sns.distplot(df_win['CRIM_wins_95%'])
sns.distplot(df_win['CRIM_wins_925%'])